In [1]:
%pip install yake
%pip install acora
%pip install uuid
%pip install find_job_titles
%pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dataclasses import dataclass, field
# from common.utils import load_programming_languages, load_stopwords, load_design_tools, load_business_analyst_skills
from acora import AcoraBuilder
import uuid
import yake
from yake.highlight import TextHighlighter
from find_job_titles import FinderAcora
from typing import Union, Tuple
import re
from PyPDF2 import PdfReader
import os
import json

In [3]:
# Get the current working directory
current_directory = os.getcwd()
print("Current directory:", current_directory)
predefined_skills_file_path = os.path.join(current_directory, "predefined_skills.txt")

def load_predefined_skills():
    with open(predefined_skills_file_path, "r") as f:
        predefined_skills = f.read().splitlines()
    return predefined_skills

skills_file_path = os.path.join(current_directory, "skills.txt")

def load_skills():
    with open(skills_file_path, "r") as f:
        skills = f.read().splitlines()
    return skills

stopwords_file_path = os.path.join(current_directory, "stopwords.txt")
def load_stopwords():
    with open(stopwords_file_path, "r") as f:
        stopwords = f.read().splitlines()
    return stopwords


Current directory: /Users/oliviazmxy/job description keyword extraction


In [4]:
def load_yake_extractor(
    language="en",
    max_ngram_size=3,
    deduplication_threshold=0.9,
    deduplication_algo="seqm",
    num_keywords=20,
) -> yake.KeywordExtractor:
    # hardcode parameters into yake extractor (change if needed later)
    return yake.KeywordExtractor(
        lan=language,
        n=max_ngram_size,
        dedupLim=deduplication_threshold,
        dedupFunc=deduplication_algo,
        top=num_keywords,
        features=None,
    )

In [5]:

def extract_job_description_yake_keywords(job_description, yake_extractor) -> list:
    stopwords = load_stopwords()

    # getting keywords using yake extractor
    raw_yake_output = yake_extractor.extract_keywords(job_description)
    yake_keywords = [
        keyword_tuple[0] for keyword_tuple in raw_yake_output
    ]  # not including score in tuple

    # removing duplicates, stopwords, and target company name (if exists)
    punctuation_chars = "!\"#$%&'()*+,-./:;<=>?@[]^_`{|}~\\"
    all_keywords = [m for m in yake_keywords if m not in stopwords and m[0] not in punctuation_chars]

    # adding found keywords to class instance
    job_yake_keywords = list(set(all_keywords))

    return job_yake_keywords

def extract_job_description_skills(job_description):
    # loading skill keywords from txt files
    skills_base_dict = {
        "skill": load_skills()
#         ,"predefined_skills": load_predefined_skills()
    }
    
    # finding other keywords from job field skill txt inputs
    found_skills_dict = dict()
    found_skills_list = list()  # easy to add to yake keywords in final function
    for skill_type, skill_list in skills_base_dict.items():
        acora_search_engine = AcoraBuilder(skill_list).build()
        found_skills_dict[skill_type] = list(set([k[0] for k in acora_search_engine.findall(job_description)]))
        found_skills_list += found_skills_dict[skill_type]
    job_skills = found_skills_dict
    return found_skills_list
    
def extract_job_description_keywords(job_description, yake_extractor):
    yake_keywords = extract_job_description_yake_keywords(job_description, yake_extractor)
    skills_keywords = extract_job_description_skills(job_description)
    job_keywords = yake_keywords
    return job_keywords, skills_keywords


In [6]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Function to normalize text
def normalize_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [13]:
# Process for the first PDF
pdf_path = '/Users/oliviazmxy/job description keyword extraction/CSOAF jd1.pdf'
text = extract_text_from_pdf(pdf_path)
job_description = normalize_text(text)

ye = load_yake_extractor()
job_keywords, skills_list = extract_job_description_keywords(job_description, ye)


print("Keywords: ", job_keywords)
print("\n\n")
print("Skills: ", skills_list)


Keywords:  ['Arts Foundation', 'visually appealing website', 'development skills', 'creating intuitive', 'Community School', 'end development', 'visually appealing', 'Opportunity', 'end development skills', 'Designer', 'appealing website layouts', 'Web', 'Web Designer', 'website layouts', 'Volunteer Opportunity', 'development', 'Vibrant Arts', 'Vibrant Arts Foundation', 'appealing website', 'intuitive and visually']



Skills:  ['UI', 'Sales Process', 'Resource Development / Management', 'Figma', 'System Engineering', 'Grant Writing / Research', 'Business Development & Sales Management', 'UX', 'Fundraising', 'Canva']


In [14]:
# Function to save skills to JSON
def save_skills_to_json(skills, json_path):
    with open(json_path, 'w') as file:
        json.dump(skills, file, indent=4)
    print(f"Skills saved to {json_path}")

all_skills = job_keywords + skills_list
json_path = 'extracted_skills_csoaf_jd1.json'
save_skills_to_json(all_skills, json_path)

Skills saved to extracted_skills_csoaf_jd1.json
